In [ ]:
#import the required libraries
import numpy as np
from collections import defaultdict
from collections import Counter
from scipy.sparse import csr_matrix, find

In [ ]:
#read the file and calculate CSR matrix
def csr_read(fname, ftype="csr", nidx=1):
    r""" 
        Read CSR matrix from a text file. 
        
        \param fname File name for CSR/CLU matrix
        \param ftype Input format. Acceptable formats are:
            - csr - Compressed sparse row
            - clu - Cluto format, i.e., CSR + header row with "nrows ncols nnz"
        \param nidx Indexing type in CSR file. What does numbering of feature IDs start with?
    """
    
    with open(fname) as f:
        lines = f.readlines()
    
    if ftype == "clu":
        p = lines[0].split()
        nrows = int(p[0])
        ncols = int(p[1])
        nnz = long(p[2])
        lines = lines[1:]
        assert(len(lines) == nrows)
    elif ftype == "csr":
        nrows = len(lines)
        ncols = 0 
        nnz = 0 
        for i in xrange(nrows):
            p = lines[i].split()
            if len(p) % 2 != 0:
                raise ValueError("Invalid CSR matrix. Row %d contains %d numbers." % (i, len(p)))
            nnz += len(p)/2
            for j in xrange(0, len(p), 2): 
                cid = int(p[j]) - nidx
                if cid+1 > ncols:
                    ncols = cid+1
    else:
        raise ValueError("Invalid sparse matrix ftype '%s'." % ftype)
    val = np.zeros(nnz, dtype=np.float)
    ind = np.zeros(nnz, dtype=np.int)
    ptr = np.zeros(nrows+1, dtype=np.long)
    n = 0 
    for i in xrange(nrows):
        p = lines[i].split()
        for j in xrange(0, len(p), 2): 
            ind[n] = int(p[j]) - nidx
            val[n] = float(p[j+1])
            n += 1
        ptr[i+1] = n 
    
    assert(n == nnz)
    
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.float)
    mat.sort_indices()
    return mat

In [ ]:
fname1 = "train.dat"

In [ ]:
csr_read(fname1)

In [ ]:
mat = csr_read(fname1, ftype="csr", nidx=1)

In [ ]:
#Calculate IDF and then do L2 normalization on the same
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = float(1.0/np.sqrt(rsum))
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [ ]:
mat2 = csr_idf(mat, copy=True)
mat3 = csr_l2normalize(mat2, copy=True)

In [ ]:
print mat3.shape

In [ ]:
#choosing of two random numbers for centroid
from sklearn.utils import shuffle
import random
def inCentroids(y, k):
    y_shuffle = shuffle(y, random_state=0)
    return y_shuffle[:k,:]

In [ ]:
#calculation of cosine similarity between other points and centroid
def cos_sim(y1, y2):
    cos_sims = y1.dot(y2.T)
    return cos_sims

In [ ]:
#assigning of points to a particular centroid
def cent_find(mat, centroids):
    index = list()
    cos_simsMat = sim(mat, centroids)

    for i in range(cos_simsMat.shape[0]):
        row = cos_simsMat.getrow(i).toarray()[0].ravel()
        top_indices = row.argsort()[-1]
        top_values = row[row.argsort()[-1]]
        index.append(top_indices + 1)
    return idx

In [ ]:
#recalculation of centre of each group
def means_cal(mat, index, k):
    centroids = list()
    for i in range(1,k+1):
        dex = [j for j, y in enumerate(idx) if y == i]
        if (part.shape[0] > 1):
            centroids.append(part.toarray().mean(0))
    csr_center = csr_matrix(centroids)
    return csr_center

In [ ]:
#implementation of Kmeans algorithm
from sklearn.cluster import KMeans
def kmeans(k, mat, iterations):
    centroids = inCentroids(mat, k)
    for _ in range(iterations): 
        index = cent_find(mat, centroids)            
        centroids = means_cal(mat, index, k)        
    return index

In [ ]:
#code for bisecting kmeans implementation
def bisectingKmeans(matrix, kclusters):
    init_cluster = []
    init_cluster.append(matrix)
    clusterindex = []
    for i in range(0, matrix.shape[0]):
       
        clusterindex.append(1)
    cid = 1
    for i in range(1, 7):
       
        index_track = []
        target_list = []

        for j in range(0, len(clusterindex)):
            
            if(clusterindex[j] == cid):
                
                target_list.append(matrix[j,:].toarray()[0])
                index_track.append(j)
            elif(clusterindex[j] > cid):
                 clusterindex[j] = clusterindex[j]+1
       
        target_mat = csr_matrix(target_list)
        
        processed_index = kmeans(7,target_mat, 20)
        
        for z in range(len(processed_index)):
            
            if(processed_index[z] == 2):
                
                clusterindex[index_track[z]] = cid + 1
        
       
        temp = []
        for k in range(0, i+1):
           
            temp.append(0)
      
        for j in range(0, len(clusterindex)):
            
            clusterIndex = clusterindex[j] - 1;
            temp[clusterIndex] = temp[clusterIndex] + 1
        
        max = temp[0]
        cid = 1
       
        for k in range(1, i+1):
            if (temp[k] > max):
                
                cid = k + 1
        
    return clusterindex 

In [ ]:
#calling Bisecting kmeans
bisectingKmeans(mat3, 2)

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import calinski_harabaz_score
x_axis = list()
y_axis = list()
for k in range(3, 22, 2):
    idx = kmeans(k, mat3, 10)
    score = calinski_harabaz_score(mat3.toarray(), idx)
    print k, score
    x_axis.append(k)
    y_axis.append(score)

In [ ]:
file_t = open("format.dat", "w")
for i in idx:
    
    file_t.write(str(i) +'\n')
file_t.close()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(x_axis, y_axis )